# 19기 KNN 정규세션 과제

## KNN 구현해보기
### 1. Preprocssing / EDA
지금까지 배운 내용을 토대로 해당 데이터에 대해 자유롭게 전처리와 EDA를 진행해주세요.
### 2. KNN 구현 & 파라미터 튜닝
수업 내용 및 실습 자료를 참고하여 KNN을 구현하고 파라미터 튜닝을 하며 결과를 비교해주세요.
### 3. Evaluation
결과에 대한 평가를 진행하고, 나름의 해석을 달아주세요.

**데이터:** [blackfriday | Kaggle](https://www.kaggle.com/llopesolivei/blackfriday)

---

## 0. 데이터 불러오기

In [1]:
import pandas as pd
df = pd.read_csv("blackfriday.csv", index_col = 0)
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780


df.shape

In [4]:
# 데이터 수: 4998개

# 결측치
# Product_Category_2, 3에 결측치 존재

# 카테고리 변수 -> int형으로 되어있음(Marital_Status, Product_Category_1, Occupation)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     4998 non-null   int64  
 1   Product_ID                  4998 non-null   object 
 2   Gender                      4998 non-null   object 
 3   Age                         4998 non-null   object 
 4   Occupation                  4998 non-null   int64  
 5   City_Category               4998 non-null   object 
 6   Stay_In_Current_City_Years  4998 non-null   object 
 7   Marital_Status              4998 non-null   int64  
 8   Product_Category_1          4998 non-null   int64  
 9   Product_Category_2          3465 non-null   float64
 10  Product_Category_3          1544 non-null   float64
 11  Purchase                    4998 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 507.6+ KB


In [5]:
df.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,4.998000e+03,4998.000000,4998.000000,4998.000000,3465.000000,1544.000000,4998.000000
mean,1.003015e+06,8.036815,0.421369,5.320128,9.773737,12.766839,9234.655462
std,1.709910e+03,6.442697,0.493828,3.912281,5.101867,4.023591,4999.225081
min,1.000003e+06,0.000000,0.000000,1.000000,2.000000,3.000000,13.000000
25%,1.001560e+06,2.000000,0.000000,1.000000,5.000000,9.000000,5497.750000
50%,1.003040e+06,7.000000,0.000000,5.000000,9.000000,14.000000,8049.000000
75%,1.004417e+06,14.000000,1.000000,8.000000,15.000000,16.000000,12039.000000
max,1.006037e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23913.000000


In [12]:
df = df.drop(['Product_Category_2', 'Product_Category_3'], axis = 1)

In [20]:
df

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5780
...,...,...,...,...,...,...,...,...,...,...
4993,1000837,P00147942,M,55+,7,C,1,1,1,11701
4994,1000293,P00358242,M,55+,1,C,1,1,8,2045
4995,1001908,P00173342,M,55+,13,C,3,1,1,4505
4996,1005517,P00336742,M,55+,0,C,2,1,12,1413


In [17]:
df['Age'].value_counts()

26-35    1996
36-45    1000
18-25     906
46-50     414
51-55     350
55+       195
0-17      137
Name: Age, dtype: int64

In [18]:
df['Gender'].value_counts()

M    3765
F    1233
Name: Gender, dtype: int64

In [19]:
df['Occupation'].value_counts()

4     706
0     572
7     535
1     460
17    339
12    305
20    285
2     251
14    245
16    244
6     186
3     160
15    133
10    122
5      98
11     95
19     73
13     65
18     56
9      50
8      18
Name: Occupation, dtype: int64

In [21]:
# 카테고리형 변수들은 모두 문자열로 변환
df['Occupation'] = df['Occupation'].astype(str)
df['Product_Category_1'] = df['Product_Category_1'].astype(str)
df['Marital_Status'] = df['Marital_Status'].astype(str)

In [25]:
df.info() # stay_in_current_city_years는 양적 척도이므로 int형으로 변환해줌

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   User_ID                     4998 non-null   int64 
 1   Product_ID                  4998 non-null   object
 2   Gender                      4998 non-null   object
 3   Age                         4998 non-null   object
 4   Occupation                  4998 non-null   object
 5   City_Category               4998 non-null   object
 6   Stay_In_Current_City_Years  4998 non-null   object
 7   Marital_Status              4998 non-null   object
 8   Product_Category_1          4998 non-null   object
 9   Purchase                    4998 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 558.6+ KB


In [36]:
df = df.drop('User_ID', axis = 1)

In [40]:
df = df.drop('Product_ID', axis = 1)

In [41]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

In [42]:
y_train = train['Product_Category_1']
y_test = test['Product_Category_1']
x_train = train.drop('Product_Category_1', axis=1)
x_test = test.drop('Product_Category_1', axis=1)

In [43]:
x_train

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Purchase
1738,M,26-35,15,A,1,0,4016
3986,M,36-45,11,C,1,1,1943
2915,M,26-35,17,C,3,1,15555
1595,M,26-35,7,A,2,1,8883
3213,F,36-45,20,C,3,1,8050
...,...,...,...,...,...,...,...
4426,M,46-50,0,C,4+,1,7904
466,M,18-25,7,A,0,0,6890
3092,F,36-45,17,A,2,0,6909
3772,M,36-45,6,B,4+,0,8112


In [44]:
x_train2 = pd.get_dummies(x_train) # 더밉변수화
x_test2 = pd.get_dummies(x_test)

In [45]:
x_train2

,Purchase,Gender_F,Gender_M,Age_0-17,Age_18-25,Age_26-35,Age_36-45,Age_46-50,Age_51-55,Age_55+,...,City_Category_A,City_Category_B,City_Category_C,Stay_In_Current_City_Years_0,Stay_In_Current_City_Years_1,Stay_In_Current_City_Years_2,Stay_In_Current_City_Years_3,Stay_In_Current_City_Years_4+,Marital_Status_0,Marital_Status_1
1738,4016,0,1,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
3986,1943,0,1,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,1
2915,15555,0,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1595,8883,0,1,0,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
3213,8050,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,7904,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,1
466,6890,0,1,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
3092,6909,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,1,0
3772,8112,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,1,0


In [48]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5, p=1) #p = 1: 맨하탄 거리, p = 2: 유클리드 거리

In [51]:
model.fit(x_train2, y_train)

predict = model.predict(x_test2)

In [53]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, predict) # 정확도 계산

0.837

## GridSearch

In [54]:
from sklearn.model_selection import GridSearchCV

In [55]:
parameter = {
    "n_neighbors": [i for i in range(1, 20)],
    "p": [1, 2]
}

In [56]:
model = KNeighborsClassifier() #p = 1: 맨하탄 거리, p = 2: 유클리드 거리

In [57]:
Grid = GridSearchCV(model, param_grid = parameter)

In [58]:
Grid.fit(x_train2, y_train)

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19],
                         'p': [1, 2]})

In [59]:
Grid.best_params_

{'n_neighbors': 7, 'p': 1}

In [60]:
model2 = KNeighborsClassifier(n_neighbors = 7, p = 1)
model2.fit(x_train2, y_train)

KNeighborsClassifier(n_neighbors=7, p=1)

In [62]:
predict2 = model2.predict(x_test2)

In [63]:
accuracy_score(y_test, predict2) # 정확도 계산

0.847